<a href="https://colab.research.google.com/github/82KJ/ARS-system-for-protecting-telephone-counselor/blob/main/SH/Test_1_to_5/sample_classifier_combination_test_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본 Setting

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=688998 sha256=a7829398cdb66d506f4ca6cd308861ab336400ce49c1758d51c232d0fa349c1a
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-8ooc_i9j
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-8ooc_i9j
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
device = torch.device("cuda:0")

In [8]:
# cd /content/drive/MyDrive/ARS-Protection/ARS-system-for-protecting-telephone-counselor/SH/Test_1_to_5
%cd /content/drive/MyDrive/bert

/content/drive/MyDrive/bert


In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/bert/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/bert/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## 데이터셋 전처리 진행

In [61]:
import pandas as pd
data1 = pd.read_csv("new_preprocess_1.csv")
data2 = pd.read_csv("new_preprocess_2.csv")
data3 = pd.read_csv("new_preprocess_3.csv")
data4 = pd.read_csv("new_preprocess_4.csv")
data5 = pd.read_csv("new_preprocess_5.csv")

In [62]:
data1.head()

,Unnamed: 0,0,1
0,0,쉴드가 아니라 국가가 면제해준거야.,0
1,1,짐승입장도 들어봐야지ㅋㅋ,0
2,2,깐부라고 부르기까지 하다니,0
3,3,어머니 간 이식 해 주기 싫어서 도망쳤다고...?,0
4,4,와 드디어 일 다 했다,0


In [63]:
print(len(data1))

31162


In [64]:
data = pd.concat([data1, data2, data3, data4, data5],axis = 0)

In [65]:
data.head()

,Unnamed: 0,0,1
0,0,쉴드가 아니라 국가가 면제해준거야.,0
1,1,짐승입장도 들어봐야지ㅋㅋ,0
2,2,깐부라고 부르기까지 하다니,0
3,3,어머니 간 이식 해 주기 싫어서 도망쳤다고...?,0
4,4,와 드디어 일 다 했다,0


In [66]:
print(len(data))

173676


In [67]:
len(data.loc[data['1'] != 0]) # 비윤리 문장 개수

19403

In [68]:
temp_list = list()
for sen, lab in zip(data["0"], data["1"]):
  temp_list.append([sen,lab])

In [69]:
temp_list[:10]

[['쉴드가 아니라 국가가 면제해준거야.', 0],
 ['짐승입장도 들어봐야지ㅋㅋ', 0],
 ['깐부라고 부르기까지 하다니', 0],
 ['어머니 간 이식 해 주기 싫어서 도망쳤다고...?', 0],
 ['와 드디어 일 다 했다', 0],
 ['그동안 고생 많았어', 0],
 ['고생한 기념으로 내가 저녁 사 줄게', 0],
 ['한 번 보여줘봐', 0],
 ['저렇게 운전해 놓고 내려서 자기가 화를 내는거냐?', 0],
 ['저런 대응은 정말 이해가 안되지?', 0]]

In [70]:
x = 0
data_list = list()
for i in range(len(temp_list)):
  if temp_list[i][1] == 0 and x < 20000:
    data_list.append(temp_list[i])
    x += 1
  elif temp_list[i][1] !=0 :
    data_list.append(temp_list[i])

In [75]:
len(data_list)

39403

In [76]:
for i in range(len(data_list)):
  if data_list[i][1] == 2:
    data_list[i][1] = 1
  elif data_list[i][1] == 3:
    data_list[i][1] = 2

In [77]:
a = 0
b = 0
c = 0
for x in data_list:
  if x[1] == 0:
    a += 1
  elif x[1] == 1:
    b+= 1
  elif x[1] == 2:
    c += 1
a,b,c

(20000, 13203, 6200)

## Train data, Test data 분리

In [78]:
from sklearn.model_selection import train_test_split

In [79]:
train_set, test_set = train_test_split(data_list, test_size=0.2, random_state=0) # train : test = 9 : 1

In [80]:
len(train_set), len(test_set)

(31522, 7881)

In [81]:
train_set[0]

['개한테 대줬을 수도 있음', 2]

## KoBERT 입력 데이터 구성하기
BERT 입력값은 (token + segment + position embedding의 합으로 구성)  
https://happy-obok.tistory.com/23 여기서 확인!!

In [82]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):

    # sentence , label data를 BERT의 입력값에 맞게 변환하는 transformer를 생성
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)

    ## 생성한 transformer로 sentence를 변환하여 저장
    self.sentences = [transform([data[sent_idx]]) for data in dataset]
    self.labels = [np.int32(data[label_idx]) for data in dataset]
  
  def __getitem__ (self, i):
    return (self.sentences[i] + (self.labels[i], )) # 각 index에 맞는 item 반환 진행 --> 왜 이런 형태인지는 잘 모르겠음
  
  def __len__(self):
    return (len(self.labels))
    

In [84]:
# Parameter setting 진행
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [85]:
# Kobert 모듈에서 제공하는 get_tokenizer와 vocab를 활용해 tokneizer를 구성한다
tokenizer = get_tokenizer() 
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

using cached model. /content/drive/MyDrive/bert/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [86]:
data_train[0]

## 출력값 설명
## 1번 array - 1로 padding된 sequence - 총 길이 64 고정
## 2번 array - padding 전 sequnce 길이 - 42
## 3번 array - 어텐션 마스크 sequence (1로 padding이 된 부분은 어텐션 함수가 적용되지 않아도 됨을 알려주는 시퀀스)
##           - 일단은 전부 0으로 구성하고, 아래의 gen_attention_mask에서 필요한 부분만 1로 변경
## 마지막 숫자 - label값

(array([   2,  835, 7835, 1633, 7305, 7088, 2874, 3883,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(9, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [87]:
# Torch에 맞게 최종 입력 dataset 구성 

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## KoBert 다중 분류기를 위한 미세 조정 모델 만들기


In [91]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size = 768, num_classes=3, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    ## classifier는 선형 회귀 모델로 구성 (input size = 768, output size = 4 (label이 4개로 구성))
    self.classifier = nn.Linear(hidden_size, num_classes)

    ## overfitting 방지를 위한 dropout 비율 설정
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  # attention mask sequence를 구성해주는 함수 --> padding이 아닌 영역을 0에서 1로 변경
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i,v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    
    return attention_mask.float()
  
  # bert + classifier를 관통하는 forward 연산 진행
  def forward(self, token_ids, valid_length, segment_ids):

    # attention_mask 계산
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    # bert에 input 투입, 변수명이 pooler인거 보니 출력 embedding에 mean pooling 적용한 값이지 않을까 추측
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

    # dropout 비율이 존재한다면, dropout 적용
    if self.dr_rate:
        out = self.dropout(pooler)

    # classifier 진행
    return self.classifier(out) 

In [92]:
# Kobert + classifier 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer (Adam), scheduler 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [93]:
for e in range(num_epochs):

    train_acc = 0.0
    test_acc = 0.0
    model.train()

    # train set
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # forward 연산 진행
        out = model(token_ids, valid_length, segment_ids)

        # loss 는 CrossEntropyLoss를 이용 -> backpropagation 학습 진행
        loss = loss_fn(out, label)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # train data set 정확도 확인
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()

    # test set
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        
        # test set 정확도 확인
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-93-26ba33054b28>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/493 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.2973698377609253 train acc 0.140625
epoch 1 batch id 201 loss 0.047539129853248596 train acc 0.8066697761194029
epoch 1 batch id 401 loss 0.13846777379512787 train acc 0.8909756857855362
epoch 1 train acc 0.906412197231834


<ipython-input-93-26ba33054b28>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/124 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8375476030465949


  0%|          | 0/493 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0025163295213133097 train acc 1.0
epoch 5 batch id 201 loss 0.0012876519467681646 train acc 0.9957245024875622
epoch 5 batch id 401 loss 0.0011001467937603593 train acc 0.9967269326683291
epoch 5 train acc 0.9970524847870182


  0%|          | 0/124 [00:00<?, ?it/s]

epoch 5 test acc 0.8367915546594982


## 학습된 Model로 결과 예측해보기

In [94]:
def predict(predict_sentence):

    # 1. data set 구성 (문장, 라벨)
    data = [predict_sentence, '0']
    dataset_another = [data]

    # 2. data를 bert의 입력에 맞게 변환하기
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # 모델 forward 연산 진행
        out = model(token_ids, valid_length, segment_ids)

        # torch out -> numpy 형식으로 변환
        test_eval=[]
        logits = out[0].detach().cpu().numpy()
        
        # 값이 가장 큰 인덱스를 출력
        if np.argmax(logits) == 0:
            test_eval= "일반"
        elif np.argmax(logits) == 1:
            test_eval = "폭언"
        elif np.argmax(logits) == 2:
            test_eval = "성희롱"
        # elif np.argmax(logits) == 3:
        #     test_eval = "성희롱"

        print(">> " + test_eval + " 문장입니다.")
     

In [95]:
while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "q":
      break
    
    predict(sentence)
    print()
     

하고싶은 말을 입력해주세요 : 시발
>> 폭언 문장입니다.

하고싶은 말을 입력해주세요 : 씨발
>> 폭언 문장입니다.

하고싶은 말을 입력해주세요 : 시발 나한테 죽고 싶냐
>> 폭언 문장입니다.

하고싶은 말을 입력해주세요 : 시발 좃같네
>> 폭언 문장입니다.

하고싶은 말을 입력해주세요 : 씨발 좃같네
>> 폭언 문장입니다.

하고싶은 말을 입력해주세요 : 니 지금 나랑 장난치냐
>> 일반 문장입니다.

하고싶은 말을 입력해주세요 : 이게 일처리를 제대로 하는거냐 너 미침?
>> 일반 문장입니다.

하고싶은 말을 입력해주세요 : 너 미쳤냐
>> 일반 문장입니다.

하고싶은 말을 입력해주세요 : q


In [96]:
torch.save(model.state_dict(),'new_preprocess_ver2.pth')


In [97]:
model.load_state_dict(torch.load('new_preprocess_ver2.pth'))

<All keys matched successfully>